In [77]:
import numpy as np
import h5py
import torch
#from torch.utils.data import Dataset, DataLoader
# import torch.utils.data as data
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

In [78]:
# helper functions

# make minibatch for training
def minibatch(X, y, batch_size=50):
    X = X.data.numpy()
    y = y.data.numpy()
    N = X.shape[0]
    batch_mask = np.random.choice(N, batch_size)
    X_batch = X[batch_mask]
    y_batch = y[batch_mask]
    return Variable(torch.Tensor(X_batch)), Variable(torch.Tensor(y_batch))

Helper Clases

In [79]:
class EEGDataset(data_utils.Dataset):
    """EEG dataset."""
    def __init__(self, X, y):
        self.X = X;
        self.y = y;
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
class Flatten(nn.Module):
    def forward(self, x):
        N, E, T = x.size() 
        return x.view(N, -1)

Load Data

In [80]:
A01T = h5py.File('project_datasets/A01T_slice.mat', 'r')
print (A01T)
X = np.copy(A01T['image'])
y = np.copy(A01T['type'])
X = X[:, 0:23, :] # only goes to 23 since you dont use last 3 channels
# X = np.moveaxis(X, -1, 0)
y = y[0,0:X.shape[0]:1]
#y = np.asarray(y, dtype=np.int32)
#subtract 769 so classes start at 0
y = y - 769
print (np.shape(X))

<HDF5 file "A01T_slice.mat" (mode r)>
(288, 23, 1000)


In [100]:
data = data_utils.TensorDataset(torch.Tensor(X), torch.Tensor(y))
dset = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [len(data)-100, 50, 50])
dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=10, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}

In [72]:
X['train'], X['val'], X['test'] = random_split(X, [len(X)-100, 50, 50])
y['train'], y['val'], y['test'] = random_split(y, [len(y)-100, 50, 50])
train = data_utils.TensorDataset(X['train'], y['train'])
val = data_utils.TensorDataset(X['val'], y['val'])
test = data_utils.TensorDataset(X['test'], y['test'])
#num_workers for GPU stuff
#dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=10, shuffle=True, num_workers=1) for x in ['train', 'val', 'test']}
dataloaders = {x: data_utils.DataLoader(x, batch_size=10, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}

channels = 23
classes = 4
#for i_batch, sample_batched in enumerate(dataloaders['train']):
#    X, y = sample_batched
#    print (X)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [101]:
dtype = torch.DoubleTensor
model = nn.Sequential(
    Flatten(),
    nn.Linear(23*1000, 1024),
    nn.ReLU(inplace=True),
    nn.Linear(1024, 4),
)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [102]:
#training

num_epochs = 10
batch_size = 10

#iterations_per_epoch = N//batch_size
best_val_acc = 0
train_acc_at_best_val = 0

#for epoch in range(1):
for epoch in range(num_epochs):
    #for t in range(iterations_per_epoch):
    for i, sample_batched in enumerate(dataloaders['train'], 0):
        X_train, y_train = sample_batched
        # Flip axis first
        #X_train = torch.Tensor(np.moveaxis(X_train.numpy(), -1, 0))
        X_train = Variable(X_train).type(dtype)
        y_train = Variable(y_train).long()
        # Calculate the loss
        y_pred = model(X_train)
        print (y_pred)
        print (y_train)
        loss = loss_fn(y_pred, y_train)
        print('(%d batch) loss: %f' % (i, loss))
        # Do backpropagation
        model.zero_grad()
        loss.backward()
        for f in model.parameters():
            print(f.grad)
        # Update the parameters
        optimizer.step()    

Variable containing:
 2.4470  0.2310  1.1605  2.1201
 0.5056  3.7127  0.6209  0.0492
 2.8441  0.0677 -2.9403 -0.4424
-1.4628 -0.5382  0.5006 -1.3143
 0.0376 -5.4875 -2.4373 -4.0100
-2.5368  0.4413  0.8446  0.2307
 3.6333 -0.8781 -0.5314 -1.6676
 0.2658  0.1288  0.0569  1.3352
-0.4382 -0.5942  0.0718 -2.5512
 0.6392 -0.0416  5.4198  0.4154
[torch.DoubleTensor of size 10x4]

Variable containing:
 0
 2
 3
 3
 3
 0
 3
 2
 1
 0
[torch.LongTensor of size 10]

(0 batch) loss: 3.149548
Variable containing:
-0.0271 -0.0340 -0.0111  ...   0.0065 -0.0072 -0.0000
 0.1500  0.1901  0.1519  ...   0.0129 -0.0308  0.0544
-0.0183 -0.0318 -0.0356  ...   0.0088  0.0057 -0.0046
          ...             ⋱             ...          
-0.0214 -0.0132  0.0077  ...   0.0173 -0.0217  0.0032
 0.0445  0.0501  0.0243  ...   0.0368 -0.0010  0.0208
-0.0412 -0.0619 -0.0255  ...   0.0134  0.0245  0.0233
[torch.DoubleTensor of size 1024x23000]

Variable containing:
1.00000e-02 *
 -0.1936
  0.3811
 -0.3662
    ⋮   
 -0.18

Variable containing:
 -29.6349   51.2386   11.4969  -58.4939
 -75.4798   20.1333   59.6354  -28.5925
-310.9972  144.2007  -45.9827  207.4850
 -14.7970 -136.7557   91.7281  131.1538
 -30.0457  -88.9119   48.1895   90.4709
 -85.0348  -60.3306   79.1210   90.0113
-218.4144  -41.2497  240.8568   56.5702
-139.1765  113.3148  -13.0451   27.1853
-186.2964   67.6582   55.5386   70.6097
  -2.2346  -71.0239   84.7316    5.2531
[torch.DoubleTensor of size 10x4]

Variable containing:
 2
 0
 0
 0
 2
 3
 2
 2
 2
 0
[torch.LongTensor of size 10]

(6 batch) loss: 111.001942
Variable containing:
-0.0062  0.0001  0.0032  ...  -0.0075  0.0025  0.0000
-0.0108 -0.0301 -0.0068  ...  -0.0274 -0.0059 -0.0083
-0.0168 -0.0094 -0.0164  ...   0.0170  0.0047 -0.0028
          ...             ⋱             ...          
-0.0350 -0.0262 -0.0333  ...  -0.0357 -0.0051 -0.0102
-0.0033  0.0191  0.0138  ...  -0.0279 -0.0375 -0.0439
 0.0030  0.0063  0.0049  ...  -0.0057 -0.0057 -0.0221
[torch.DoubleTensor of size 1024x230

Variable containing:
  71.6679  -92.0596   -5.8659   42.5161
  52.1882   96.1024  -21.1628 -121.3085
 140.8951  148.8328 -239.0217  -69.4565
  44.1994   46.5102 -103.2768   -0.6396
-134.3176   22.8399    7.0443   75.7059
 264.6188  160.3332 -327.5982   10.6479
 -10.2524   40.7338  -27.1119  -38.1033
 -17.8446  -30.8570  120.2041 -111.1553
   6.9613  -38.9635  -20.8084    7.1104
 -90.4727  -15.9209  208.7453  -77.6338
[torch.DoubleTensor of size 10x4]

Variable containing:
 3
 0
 1
 1
 2
 1
 0
 3
 3
 2
[torch.LongTensor of size 10]

(12 batch) loss: 52.907505
Variable containing:
-0.0074 -0.0026 -0.0081  ...  -0.0003 -0.0010 -0.0018
-0.0219 -0.0604 -0.0374  ...   0.0088  0.0153  0.0131
-0.0148 -0.0436 -0.0261  ...   0.0065  0.0114  0.0099
          ...             ⋱             ...          
-0.0469 -0.0473 -0.0568  ...   0.0003  0.0016 -0.0014
-0.0077 -0.0073 -0.0005  ...   0.0196  0.0109 -0.0065
 0.0151  0.0176  0.0190  ...  -0.0006 -0.0014 -0.0004
[torch.DoubleTensor of size 1024x230

Variable containing:
-0.0025 -0.0014 -0.0025  ...   0.0017 -0.0003  0.0004
 0.0165 -0.0329 -0.0312  ...  -0.0358  0.0325 -0.0269
 0.0131 -0.0251 -0.0241  ...  -0.0290  0.0241 -0.0209
          ...             ⋱             ...          
 0.0179  0.0097  0.0178  ...  -0.0119  0.0018 -0.0027
 0.0110 -0.0201 -0.0270  ...  -0.0027  0.0333  0.0121
-0.0064  0.0160  0.0141  ...   0.0114 -0.0182  0.0118
[torch.DoubleTensor of size 1024x23000]

Variable containing:
1.00000e-02 *
 -0.0523
  0.3906
  0.3201
    ⋮   
  0.3740
  0.4739
 -0.1131
[torch.DoubleTensor of size 1024]

Variable containing:
 32.0326   2.6946   6.5557  ...   15.2695   9.5705  18.2429
-32.0326  -6.4732  -8.8352  ...  -15.2695 -13.1185 -15.5551
  0.0000   3.7786   2.2796  ...    0.0000   3.5480  -2.6879
  0.0000   0.0000   0.0000  ...    0.0000   0.0000   0.0000
[torch.DoubleTensor of size 4x1024]

Variable containing:
 0.0997
-0.1997
 0.0000
 0.1000
[torch.DoubleTensor of size 4]

Variable containing:
 158.0268  156.1060   3

Variable containing:
  -5.2176   51.5618   20.0622  -73.3819
 -74.9699  486.6803 -104.8361 -395.1873
 -71.5218  -63.7402   84.6371  330.6966
  33.1613   79.3080   46.1290 -119.0221
 -97.9665   48.0025  -18.2507   30.4878
 193.6007   73.8619 -263.9297   71.3178
  32.1467   11.1627    0.3754  -44.9315
 -21.1447  -27.0135  137.2676   18.9282
 -21.5252   39.0973   12.5914  -21.2931
 -92.6103  305.9882  -11.3176 -124.1066
[torch.DoubleTensor of size 10x4]

Variable containing:
 2
 1
 3
 0
 3
 0
 2
 2
 2
 0
[torch.LongTensor of size 10]

(4 batch) loss: 55.203676
Variable containing:
-0.0178 -0.0126 -0.0066  ...   0.0178  0.0065  0.0069
-0.0087  0.0171  0.0205  ...   0.0217  0.0155  0.0109
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
-0.0063 -0.0081  0.0008  ...  -0.0155  0.0098  0.0024
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
[torch.DoubleTensor of size 1024x2300

Variable containing:
-0.0076 -0.0054 -0.0044  ...   0.0050  0.0030  0.0031
 0.0000  0.0000  0.0000  ...  -0.0000 -0.0000 -0.0000
 0.0418  0.0283  0.0309  ...  -0.0072  0.0057  0.0057
          ...             ⋱             ...          
 0.0207  0.0140  0.0153  ...  -0.0036  0.0028  0.0028
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
-0.0205 -0.0139 -0.0151  ...   0.0035 -0.0028 -0.0028
[torch.DoubleTensor of size 1024x23000]

Variable containing:
1.00000e-03 *
  0.9753
 -0.0000
 -2.9399
    ⋮   
 -1.4566
  0.0000
  1.4400
[torch.DoubleTensor of size 1024]

Variable containing:
-10.1772   0.0000  -5.8865  ...   -7.9899   0.0000  -5.0668
 10.1772   0.0000   5.8865  ...    7.9899   0.0000   5.0668
  1.2696   0.0000   0.0000  ...    0.0000   0.0000   0.0000
 -1.2696   0.0000   0.0000  ...    0.0000   0.0000   0.0000
[torch.DoubleTensor of size 4x1024]

Variable containing:
-0.1993
 0.1000
 0.1000
-0.0007
[torch.DoubleTensor of size 4]

Variable containing:
-152.4307  -30.7834  15

Variable containing:
 -22.0678  -30.4840   87.3876  -48.0495
 -41.8530   51.3805   18.7472    2.0991
  69.7881   -3.0852   94.2342  -57.7108
  30.6661  635.2592 -172.7437  -75.8196
 142.8267  229.4195  128.4761 -212.0394
 -42.5296  -38.4355  206.4450  -99.5338
 165.7044  -59.5533   98.4258  -95.2466
 -45.5068   -8.0927   58.8102    6.3172
-143.6219  229.7155  124.2511 -216.3857
  34.9987  -19.8495  -22.9202   54.9881
[torch.DoubleTensor of size 10x4]

Variable containing:
 2
 1
 1
 1
 0
 2
 0
 1
 1
 3
[torch.LongTensor of size 10]

(15 batch) loss: 25.081514
Variable containing:
-0.0031 -0.0048 -0.0041  ...   0.0017  0.0025  0.0009
 0.0389  0.0596  0.0512  ...  -0.0215 -0.0304 -0.0107
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
-0.0048 -0.0058 -0.0026  ...   0.0017 -0.0023 -0.0020
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
-0.0095 -0.0114 -0.0050  ...   0.0033 -0.0046 -0.0039
[torch.DoubleTensor of size 1024x230

Process Process-952:
Process Process-950:
Process Process-951:
Process Process-949:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/wilsonchang/Des

KeyboardInterrupt: 

In [9]:
y_pred = model(X_train)
loss = loss_fn(last_output, y_train)
print (loss)
model

NameError: name 'last_output' is not defined